In [3]:
import numpy as np
import torch
import sys
sys.path.append('../')
from voting_games.werewolf_env_v0 import pare, pare_Role, pare_Phase
import random
from tqdm import tqdm

In [25]:
env = pare(num_agents=10, werewolves=2)
env.reset()
num_games = 10

villager_wins = 0
avg_game_length = 0

for _ in range(num_games):
    env.reset()

    while env.agents:
        actions = {}
        for agent in env.agents:
            if env.agent_roles[agent] == pare_Role.VILLAGER and env.world_state["phase"] != pare_Phase.NIGHT:
                actions[agent] = env.action_space(agent).sample().tolist()
            elif env.agent_roles[agent] == pare_Role.WEREWOLF:
                actions[agent] = env.action_space(agent).sample().tolist()

        observations, rewards, terminations, truncations, infos = env.step(actions)
    
    winner = env.world_state['winners']
    day = env.world_state['day']

    if winner == pare_Role.VILLAGER:
        villager_wins += 1
    
    avg_game_length += (day * 1.0)/num_games     

print(f'Average game length = {avg_game_length:.2f}')
print(f'Villager wins: {villager_wins}')

Average game length = 4.40
Villager wins: 5
